In [36]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
from sklearn.model_selection import train_test_split

In [37]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_k(x,k):
    return 1 / (1 + k*np.exp(-z))

In [38]:
def relu(x):
    return np.log(1+np.exp(x))

In [39]:
#out_layer_act
def drelu(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

In [40]:
def loss(out,Y):
        loss = (-1)*(np.sum(np.multiply(np.log(out), Y) + np.multiply((1 - Y), np.log(1 - out))))/(Y.shape[1])
        #print(loss)
        return loss

In [41]:
#testimg relu and sigmoid
#relu(5)
#relu(-3)
#relu(0)

#sigmoid(5)
#sigmoid(0)
#sigmoid(-4)

In [51]:
df_proc = pd.read_csv("preprocessed.csv") 

In [52]:
df_proc.columns=["sl_no","id","thickness", "cellsize", "cellshape", "madhesion", "secellsize", "barenuclei", "chromatin", "nucleoli", "mitosis","Class"]

In [53]:
Y = df_proc.filter(['Class'], axis=1)
X = df_proc.drop(['sl_no','id','Class'],axis=1)

In [54]:
X.shape
XTrain,XTest,YTrain,YTest=train_test_split(X, Y, test_size=0.20, random_state=42)
X,Y=X.to_numpy(),Y.to_numpy()
XTrain,XTest,YTrain,YTest= XTrain.to_numpy().T,XTest.to_numpy().T,YTrain.to_numpy().reshape(1, YTrain.shape[0]),YTest.to_numpy().reshape(1, YTest.shape[0])
X,Y=X.T,Y.reshape(1, Y.shape[0])

print(X.shape,Y.shape,XTrain.shape,YTrain.shape,XTest.shape,YTest.shape)

(9, 699) (1, 699) (9, 559) (1, 559) (9, 140) (1, 140)


In [55]:
print(X.shape,Y.shape)


(9, 699) (1, 699)


In [56]:
#taking all 9 layers as input
in_layer_no=X.shape[0] #no of attributes
hid_layer_no=int(1.5*in_layer_no)
out_layer_no=1

#initial weights
wh = np.random.randn(hid_layer_no,in_layer_no) * 0.01
bh = np.zeros(shape=(hid_layer_no, 1))
w_out = np.random.randn(out_layer_no,hid_layer_no) * 0.01
b_out = np.zeros(shape=(out_layer_no, 1))

initial_weights=[wh,bh,w_out,b_out]

In [57]:
print(in_layer_no)
print(hid_layer_no)
print(out_layer_no)
wh.shape

9
13
1


(13, 9)

In [72]:
def ann(X,Y,learning_rate):
    #taking all 9 layers as input
    in_layer_no=X.shape[0] #no of attributes
    hid_layer_no=int(1.5*in_layer_no)
    out_layer_no=1

    #initial weights
    wh = np.random.randn(hid_layer_no,in_layer_no) * 0.01
    bh = np.zeros(shape=(hid_layer_no, 1))
    w_out = np.random.randn(out_layer_no,hid_layer_no) * 0.01
    b_out = np.zeros(shape=(out_layer_no, 1))

    initial_weights=[wh,bh,w_out,b_out]
    dwh_old=0
    dw_out_old=0
    for i in range(0,100000):
        #forward propogation
        #input to hidden layer = dot product(X,wh) + bh
        hid_layer_input = np.dot(wh,X) + bh
        hid_layer_act = relu(hid_layer_input)
        
        # Final output layer prediction
        out_layer_input = np.dot(w_out,hid_layer_act) + b_out
        out_layer_act = sigmoid(out_layer_input)
        lo=loss(out_layer_act,Y)
        if(i%1000==0):
            print(i,lo)
        '''dZ2 = out_layer_act - Y
        dW2 = (1 /X.shape[1]) * np.dot(dZ2, hid_layer_act.T)
        db2 = (1 / X.shape[1]) * np.sum(dZ2, axis=1, keepdims=True)
        dZ1 = np.multiply(np.dot(w_out.T, dZ2), 1 - np.power(hid_layer_act, 2))
        dW1 = (1 / X.shape[1]) * np.dot(dZ1, X.T)
        db1 = (1 / X.shape[1]) * np.sum(dZ1, axis=1, keepdims=True)
        
        wh = wh - learning_rate * dW1
        w_out = w_out - learning_rate * dW2
        bh = bh - learning_rate * db1
        b_out = b_out - learning_rate * db2
        '''
        #backward propogation output layer    
        t_o = out_layer_act - Y
        sigmak = t_o * sigmoid(out_layer_input)*(1-sigmoid(out_layer_input))   
        dLoss_W2 = (1/hid_layer_act.shape[1]) * np.dot(sigmak,hid_layer_act.T)
        dLoss_b2 = (1/hid_layer_act.shape[1]) * np.dot(sigmak, np.ones([sigmak.shape[1],1])) 
          
        #backward propogation input layer
        dLoss_A1 = np.dot(w_out.T,sigmak)
        dLoss_Z1 = dLoss_A1 * drelu(hid_layer_input)        
        dLoss_A0 = np.dot(wh.T,dLoss_Z1)
        dLoss_W1 = 1/X.shape[1] * np.dot(dLoss_Z1,X.T)
        dLoss_b1 = 1/X.shape[1] * np.dot(dLoss_Z1, np.ones([dLoss_Z1.shape[1],1]))  
        
        wh = wh - learning_rate * dLoss_W1
        w_out = w_out - learning_rate * dLoss_W2
        bh = bh - learning_rate * dLoss_b1
        b_out = b_out - learning_rate * dLoss_b2
    return [wh,bh,w_out,b_out]
    ''' 
        #backpropogation
        dout_layer_act = (out_layer_act - Y) / (out_layer_act * (1 - out_layer_act))
        dZ2 = np.multiply(dout_layer_act, out_layer_act * (1 - out_layer_act))
        dw_out = np.dot(dZ2, hid_layer_act.T)
        #dw_out=dw_out+0.5*dw_out_old
        #dw_out_old=dw_out
        db_out = np.sum(dZ2, axis=1, keepdims=True)

        dhid_layer_act = np.dot(w_out.T, dZ2)
        dZ1 = np.multiply(dhid_layer_act, dhid_layer_act * (1 - dhid_layer_act))
        dwh = np.dot(dZ1, X.T)
        #dwh=dwh+0.2*dwh_old
        #dwh_old=dwh
        dbh = np.sum(dZ1, axis=1, keepdims=True)
        
        wh = wh - learning_rate * dwh
        w_out = w_out - learning_rate * dw_out
        bh = bh - learning_rate * dbh
        b_out = b_out - learning_rate * db_out
    '''
        
        

In [73]:
#final forward propogation to test trained model 
def predict(XTrain,YTrain,XTest,YTest,learning_rate):
    final_weights=ann(XTrain,YTrain,learning_rate)
    hid_layer_input = np.dot(final_weights[0],XTest) + final_weights[1]
    hid_layer_act = relu(hid_layer_input)
    out_layer_input = np.dot(final_weights[2],hid_layer_act) + final_weights[3]
    out_layer_act = sigmoid(out_layer_input)
    predictions = out_layer_act > 0.5
    predictions=predictions.astype(int)  
    accuracy=float((np.dot(YTest,predictions.T) + np.dot(1-YTest,1-predictions.T))/float(YTest.size)*100)
    #print(accuracy)
    #print ('Accuracy: %f' % float((np.dot(YTest,predictions.T) + np.dot(1-YTest,1-predictions.T))/float(YTest.size)*100) + '%')
    return accuracy

In [74]:
acc=predict(XTrain,YTrain,XTest,YTest,0.05)

0 0.6926638704708253
1000 0.6406218949126709
2000 0.6125360233751603
3000 0.3385779414334623
4000 0.1589472218533854
5000 0.1281541058323269
6000 0.11604351905194145
7000 0.10969276836540792
8000 0.10534936326862686
9000 0.10228024785504498
10000 0.10004864072079588
11000 0.09836706491114577
12000 0.09712558490450607
13000 0.09600335873343768
14000 0.09485658413243268
15000 0.09388672780162068
16000 0.09305552278462662
17000 0.09241967566895268
18000 0.09195482434008488
19000 0.0914442823233655
20000 0.09105855321028265
21000 0.090658380594726
22000 0.09035278713024973
23000 0.09007100161142013
24000 0.08975703528767097
25000 0.08952882933753066
26000 0.08932377804915873
27000 0.08913697941202751
28000 0.08896508156294022
29000 0.08880641298964542
30000 0.08866004404631557
31000 0.08852365551160735
32000 0.08839732399195968
33000 0.0882775455132749
34000 0.08816640073666823
35000 0.08806050576984151
36000 0.08796070637826994
37000 0.08786773135637978
38000 0.08777980582173484
39000 0.0

In [75]:
print(acc)

97.14285714285714
